# Compute particle trajectories for LLC4320 surface flow
This notebook computes trajectories and saves them to disk. `plot_LLC4320_particle_trajectories.ipynb` reads the trajectories and plots them.

Setup and import packages:

In [1]:
import xarray as xr
import numpy as np
import oceanspy as ospy
import OceInterp as oi

First open the dataset into the OceData object

In [2]:
od = ospy.open_oceandataset.from_catalog('LLC4320')
ds = od._ds
oce = oi.OceData(ds)

Opening LLC4320.
10 day hourly data from the LLC4320 simulations computed using the MITGCM, a general, curvilinear ocean simulation on the cube-sphere.
Loading grid into memory, it's a large dataset please be patient
numpy arrays of grid loaded into memory
cKD created


Initiate the particle locations randomly

In [3]:
# N = 2**17        # Number of particles. Production value
N = 2**10        # Number of particles. Test value
xx,yy,zz = [np.random.normal(size = N) for i in range(3)]
r = np.sqrt(xx**2+yy**2+zz**2)
y = (np.pi/2 - np.arccos(zz/r))*180/np.pi
x = np.arctan2(yy,xx)*180/np.pi
x = x[y>-80]
y = y[y>-80]
z = np.ones_like(x)*(-10.)       # 10m depth

We will be using just one level (the surface) of a single snapshot in time to make it faster.
This will create an ensemble of particles moving in 2D at the surface.

In [4]:
oce['u'] = oce['U'][0,0]
oce['v'] = oce['V'][0,0]
p = oi.particle(x = x,y = y,z = z,data = oce,
                 uname = 'u',vname = 'v',wname = None,
                )

The package knows that LLC4320 is a large dataset, so it doesn't prefetch data to avoid overwhelming itself. However, we need prefetching for performance and in this case the data just fits in core memory. Therefore set `p.too_large = False` to prefetch the data.

In [5]:
p.too_large = False

We are going to run it with 3 hour timestep for 30 days

In [6]:
p.t  = np.ones(p.N)*10.
step = 10800     # Step in seconds
Nsteps = 240          # Number of steps
dest = [n*step for n in range(1,Nsteps)]
p.uarray = np.array(oce['u'])
p.varray = np.array(oce['v'])

Step the Lagrangian particles. This takes a long time (e.g., several hours on SciServer).

In [7]:
raw = []
for i in range(len(dest)):
    raw.append(p.deepcopy())
    p.to_next_stop(dest[i])
    print()
raw.append(p.deepcopy())

1008 left 

/home/idies/workspace/Storage/Thomas.Haine/persistent/Poseidon testing/From_Wenrui/OceInterp/OceInterp/get_masks.py:108: UserWarning: no maskC in the dataset, assuming nothing is masked.
  warnings.warn('no maskC in the dataset, assuming nothing is masked.')
/home/idies/workspace/Storage/Thomas.Haine/persistent/Poseidon testing/From_Wenrui/OceInterp/OceInterp/get_masks.py:108: UserWarning: no maskC in the dataset, assuming nothing is masked.
  warnings.warn('no maskC in the dataset, assuming nothing is masked.')



1008 left 

/home/idies/workspace/Storage/Thomas.Haine/persistent/Poseidon testing/From_Wenrui/OceInterp/OceInterp/get_masks.py:108: UserWarning: no maskC in the dataset, assuming nothing is masked.
  warnings.warn('no maskC in the dataset, assuming nothing is masked.')


619 left 395 left 192 left 91 left 54 left 36 left 19 left 6 left 4 left 4 left 3 left 3 left 3 left 3 left 1 left 1 left 1 left 1 left 
1008 left 615 left 376 left 186 left 92 left 50 left 31 left 20 left 10 left 5 left 4 left 3 left 3 left 2 left 1 left 
1008 left 601 left 376 left 183 left 91 left 56 left 32 left 19 left 8 left 5 left 2 left 2 left 2 left 2 left 2 left 
1008 left 610 left 368 left 186 left 89 left 49 left 29 left 18 left 11 left 6 left 2 left 2 left 2 left 2 left 1 left 
1008 left 595 left 359 left 192 left 92 left 53 left 29 left 20 left 8 left 6 left 3 left 2 left 2 left 2 left 2 left 1 left 1 left 
1008 left 

/home/idies/workspace/Storage/Thomas.Haine/persistent/Poseidon testing/From_Wenrui/OceInterp/OceInterp/get_masks.py:108: UserWarning: no maskC in the dataset, assuming nothing is masked.
  warnings.warn('no maskC in the dataset, assuming nothing is masked.')


599 left 351 left 170 left 87 left 50 left 28 left 13 left 8 left 4 left 3 left 2 left 2 left 2 left 1 left 1 left 1 left 
1008 left 584 left 373 left 185 left 93 left 49 left 31 left 19 left 12 left 5 left 4 left 3 left 3 left 2 left 2 left 
1008 left 575 left 337 left 169 left 91 left 47 left 32 left 23 left 11 left 6 left 4 left 3 left 2 left 2 left 2 left 2 left 2 left 
1008 left 572 left 373 left 172 left 87 left 52 left 35 left 21 left 12 left 8 left 3 left 2 left 2 left 1 left 1 left 1 left 1 left 1 left 
1008 left 569 left 368 left 182 left 90 left 44 left 34 left 19 left 12 left 6 left 3 left 2 left 2 left 2 left 2 left 2 left 2 left 1 left 
1008 left 557 left 351 left 184 left 93 left 52 left 31 left 17 left 9 left 5 left 4 left 2 left 2 left 2 left 2 left 2 left 1 left 1 left 
1008 left 570 left 350 left 172 left 98 left 50 left 29 left 17 left 12 left 6 left 3 left 2 left 2 left 2 left 2 left 1 left 1 left 1 left 1 left 1 left 1 left 1 left 
1008 left 554 left 335 left 171 

Extract and save the data so that we don't have to do the calculation everytime

In [8]:
lons = np.array([i.lon for i in raw])
lats = np.array([i.lat for i in raw])
spds = np.array([np.hypot(i.u*i.dx,i.v*i.dy) for i in raw])
np.save('LLC4320lons.npy',lons)
np.save('LLC4320lats.npy',lats)
np.save('LLC4320spds.npy',spds)

Now run `plot_LLC4320_particle_trajectories.ipynb` to make the plots.